# This jupyter notebook shows the K-means analysis

First, Import necessary library

In [23]:
import numpy as np
import pandas as pd
import io
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors
import imgkit

Next, import the dataset and combine it into one

In [2]:
lists_0=pd.read_csv(r'grouped_lists_0.csv')
lists_1=pd.read_csv(r'grouped_lists_1.csv')
lists_2=pd.read_csv(r'grouped_lists_2.csv')
lists_3=pd.read_csv(r'grouped_lists_3.csv')
lists_4=pd.read_csv(r'grouped_lists_4.csv')
lists_1

,Unnamed: 0,Building name,Related probable/confirmed cases,count,lat_col,long_col
0,236,"Chun Yu House, Sam Shing Estate",4455,1,22.381214,113.978212
1,237,"Chung Kam House, Tin Chun Court",4171,1,37.300334,99.021896
2,238,"Chung Kam House, Tin Chung Court","3744, 3905",2,22.460450,114.000154
3,239,Chung Kin Building,4159,1,22.322871,114.160516
4,240,"Chung Lau House, Tin Chung Court","3793, 3934",2,22.460450,114.000154
...,...,...,...,...,...,...
231,467,Indi home Green River Restaurant,4413,1,NaN,NaN
232,468,Indi home Maxim's MX,4413,1,NaN,NaN
233,469,Island Lodge,4283,1,22.292321,114.201738
234,470,Java Road Municipal Services Building,"2299, 2468",2,22.292333,114.199354


In [3]:
rawdata = pd.concat([lists_0,lists_1,lists_2,lists_3,lists_4])
rawdata

,Unnamed: 0,Building name,Related probable/confirmed cases,count,lat_col,long_col,Unnamed: 0.1
0,0,1 Fu Chi Path,"1831, 2113, 3830",3,22.335799,114.207887,NaN
1,1,1 Hung To Road No. 1 Hung To Canteen,"4186, 4387",2,NaN,NaN,NaN
2,2,1 Wing Lung Street,4442,1,22.337019,114.157718,NaN
3,3,12 Po Peng Street,4338,1,3.811276,103.323790,NaN
4,4,12 Tai Hing Tai Road,4119,1,22.205907,114.027745,NaN
...,...,...,...,...,...,...,...
231,1175,"Yuk King House, King Ming Court","4297, 4298",2,22.323272,114.249775,1175.0
232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0
233,1177,Zone B Tai Po Mega Mall,"2991, 3504",2,NaN,NaN,1177.0
234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0


Drop any coordinates having null value or outside Hong Kong region(According to Hong Kong Lands Department, Hong Kong Special Administrative Region lies between Latitude 22°08' North and 22°35' North, Longitude 113°49' East and 114°31' East).

In [4]:
rawdata = rawdata.dropna()
rawdata = rawdata[rawdata.lat_col >22]
rawdata = rawdata[rawdata.lat_col <23]
rawdata = rawdata[rawdata.long_col >113]
rawdata = rawdata[rawdata.long_col <115]
rawdata

,Unnamed: 0,Building name,Related probable/confirmed cases,count,lat_col,long_col,Unnamed: 0.1
0,472,K11 MUSEA,"3369, 3441",2,22.294416,114.174843,472.0
3,475,"Ka Chun House, Ka Lung Court",3849,1,22.249281,114.138037,475.0
4,476,"Ka Lei Lau, Ka Wai Chuen","4425, 4549, 4550, 4551",4,22.307993,114.186726,476.0
6,478,"Kai Him Lau, Cho Yiu Chuen",4457,1,22.346710,114.127505,478.0
7,479,"Kai Ning House, Kai Yip Estate",3701,1,22.329837,114.209912,479.0
...,...,...,...,...,...,...,...
230,1174,"Yuet Wing House, Tin Yuet Estate","4288, 4510",2,22.462994,113.999735,1174.0
231,1175,"Yuk King House, King Ming Court","4297, 4298",2,22.323272,114.249775,1175.0
232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0
234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0


For example, there are 2 cases at building K11 MUSEA. Hence, this data should count twice during k-means analysis. When doing analysis, this location will count as two data.
Same as other building, their building name are mulitplied by count, and save in new dataframe r_append.

In [5]:
rawdata_test= rawdata.reset_index()
rawdata_test.columns
r_append =pd.DataFrame(columns = rawdata_test.columns)

for ind, count in enumerate(rawdata_test['count']):
    r_append=r_append.append([rawdata_test.loc[ind]]*count, ignore_index=True)
    print(ind)
r_append

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,index,Unnamed: 0,Building name,Related probable/confirmed cases,count,lat_col,long_col,Unnamed: 0.1
0,0,472,K11 MUSEA,"3369, 3441",2,22.294416,114.174843,472.0
1,0,472,K11 MUSEA,"3369, 3441",2,22.294416,114.174843,472.0
2,3,475,"Ka Chun House, Ka Lung Court",3849,1,22.249281,114.138037,475.0
3,4,476,"Ka Lei Lau, Ka Wai Chuen","4425, 4549, 4550, 4551",4,22.307993,114.186726,476.0
4,4,476,"Ka Lei Lau, Ka Wai Chuen","4425, 4549, 4550, 4551",4,22.307993,114.186726,476.0
...,...,...,...,...,...,...,...,...
1238,232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0
1239,232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0
1240,234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0
1241,234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0


Here we define new variables x1 and x2 to save the longitude and latitude variables.

In [6]:
x1 = r_append['long_col']
x2 = r_append['lat_col']
print('Datapoints defined!')
x1


Datapoints defined!


0       114.174843
1       114.174843
2       114.138037
3       114.186726
4       114.186726
           ...    
1238    114.238107
1239    114.238107
1240    114.171876
1241    114.171876
1242    114.165219
Name: long_col, Length: 1243, dtype: float64

Here, a variable X is introduced to save coordinates array for kMeans analysis.

In [7]:
X_tmp= pd.DataFrame(columns=('long_col', 'lat_col'))
X_tmp['long_col']=x1
X_tmp['lat_col']=x2
X= X_tmp.to_numpy()
X

array([[114.17484257,  22.2944157 ],
       [114.17484257,  22.2944157 ],
       [114.1380366 ,  22.249281  ],
       ...,
       [114.17187645,  22.2967775 ],
       [114.17187645,  22.2967775 ],
       [114.165219  ,  22.3202935 ]])

Define number of clusters kcluster as the variable and cluster the data. Different kcluster are analyzed and compared in the report.

In [49]:
kclusters=80
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=12)
k_means.fit(X)

KMeans(n_clusters=80, n_init=12)

In [50]:
k_means_labels = k_means.labels_

Define k_means_cluster_centers as the variable of cluster centers. For the following, append the value of cluster number to each of the respective data as shown in raw_test below.

A cluster dataframe clust is also created. 

In [51]:
k_means_cluster_centers = k_means.cluster_centers_
pd.DataFrame(k_means_cluster_centers)

,0,1
0,114.251015,22.304548
1,114.027082,22.443979
2,114.200176,22.347799
3,114.132115,22.502185
4,113.934511,22.278184
...,...,...
75,114.099383,22.432930
76,114.225376,22.280687
77,114.187914,22.326413
78,114.170707,22.308867


In [52]:
df = pd.DataFrame({'cluster': k_means_labels})
df

,cluster
0,60
1,60
2,44
3,35
4,35
...,...
1238,47
1239,47
1240,60
1241,60


In [53]:
raw_test = r_append
raw_test = raw_test.join(df)
raw_test

,index,Unnamed: 0,Building name,Related probable/confirmed cases,count,lat_col,long_col,Unnamed: 0.1,cluster
0,0,472,K11 MUSEA,"3369, 3441",2,22.294416,114.174843,472.0,60
1,0,472,K11 MUSEA,"3369, 3441",2,22.294416,114.174843,472.0,60
2,3,475,"Ka Chun House, Ka Lung Court",3849,1,22.249281,114.138037,475.0,44
3,4,476,"Ka Lei Lau, Ka Wai Chuen","4425, 4549, 4550, 4551",4,22.307993,114.186726,476.0,35
4,4,476,"Ka Lei Lau, Ka Wai Chuen","4425, 4549, 4550, 4551",4,22.307993,114.186726,476.0,35
...,...,...,...,...,...,...,...,...,...
1238,232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0,47
1239,232,1176,"Yuk Mei House, Yau Chui Court","3694, 3908",2,22.300251,114.238107,1176.0,47
1240,234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0,60
1241,234,1178,iSQUARE,"3941, 4001",2,22.296778,114.171876,1178.0,60


In [54]:
#create dataset for cluster center
clust = pd.DataFrame(k_means_cluster_centers)
clust = clust.reset_index()
clust = clust.rename(columns={"index":"cluster", 0: "long_col", 1: "lat_col"})
clust

,cluster,long_col,lat_col
0,0,114.251015,22.304548
1,1,114.027082,22.443979
2,2,114.200176,22.347799
3,3,114.132115,22.502185
4,4,113.934511,22.278184
...,...,...,...
75,75,114.099383,22.432930
76,76,114.225376,22.280687
77,77,114.187914,22.326413
78,78,114.170707,22.308867


Plot a folium map to show the cluster and cluster centers.

In [55]:

# create map
latitude =22.349
longitude = 114.169
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, count in zip(raw_test['lat_col'], raw_test['long_col'], raw_test['Building name'], raw_test['cluster'],raw_test['count']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

for lat, lon, cluster in zip(clust['lat_col'], clust['long_col'], clust['cluster']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='black',
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)    
map_clusters.save('kmean_80.html')
map_clusters

Below is just a quick view on the general cases in Hong Kong.

In [13]:
# plot general map
latitude =22.319
longitude = 114.169
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi in zip(rawdata['lat_col'], rawdata['long_col'], rawdata['Building name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters